In [1]:
import pandas as pd
import pyodbc
from secret import user_name

In [2]:
#db connect
server_name = 'DESKTOP-5CNJJCA\SQLEXPRESS'

user_name = user_name

conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-5CNJJCA\SQLEXPRESS;'
                      'Database=First_Database;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()

In [3]:
cursor = conn.cursor()
# select 26 rows from SQL table to insert in dataframe.
query = "SELECT * FROM cars;"
df = pd.read_sql(query, conn)
print(df.head(26))

h:\Python project\BS4 scrapers\Scrape class\venv\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


                                                title  year   price  milage  \
0                        Audi A3 1.9  TDI  Sport  2dr  2008  3850.0  105000   
1                Audi A5 1.8T  FSI  160  S  Line  2dr  2009  3995.0  132000   
2                          Audi TT 1.8  T  2dr  [163]  2006  3995.0   65000   
3                       Audi A3 1.9  TDIe  Sport  5dr  2009  3995.0  109452   
4                 Audi A4 2.0  TDi  170  S  Line  5dr  2007  3995.0  117000   
5                 Audi A6 2.0  TDI  TDV  S  Line  4dr  2008  3995.0  114000   
6    Audi A6 2.0  TDI  170  S  Line  Special  Ed  4dr  2010  4195.0  159000   
7                          A6 2.0  TDIe  S  Line  4dr  2009  4250.0  149000   
8   Audi A4 2.0  TDI  170  S  Line  4dr  [Start  S...  2009  4250.0  158000   
9                       Audi A3 1.9  TDIe  Sport  3dr  2009  4250.0  145166   
10       Audi A3 1.4  TFSI  Sport  5dr  [Start  Stop]  2012  4400.0  120000   
11                Audi A4 2.0  TDI  143  S  Line  4d

In [4]:
df['make'] = 'Audi'
df['model'] = ''

In [5]:
df.drop_duplicates(inplace=True, subset=['id'], keep='last')
df = df[df['price'] !=0]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11127 entries, 27 to 116959
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         11127 non-null  object 
 1   year          11127 non-null  int64  
 2   price         11127 non-null  float64
 3   milage        11127 non-null  int64  
 4   transmission  11127 non-null  object 
 5   fuel_type     11127 non-null  object 
 6   body_style    11127 non-null  object 
 7   engine_size   11127 non-null  float64
 8   doors         11127 non-null  int64  
 9   location      11127 non-null  object 
 10  id            11127 non-null  object 
 11  link          11127 non-null  object 
 12  timestamp     11127 non-null  object 
 13  make          11127 non-null  object 
 14  model         11127 non-null  object 
dtypes: float64(2), int64(3), object(10)
memory usage: 956.2+ KB


In [6]:
models = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'TT', 'Q2', 'Q3', 'Q4','Q5','Q6', 'Q7', 'Q8','S3','S4','RS3','RS4','RS5','RS6','RS7','R8','E-Tron']

def add_model(model):
    a6 = df['title'].str.contains(model)
    df.loc[a6,'model'] = model
    print(f'{model} done')

for model in models:
    add_model(model)


A1 done
A2 done
A3 done
A4 done
A5 done
A6 done
A7 done
A8 done
TT done
Q2 done
Q3 done
Q4 done
Q5 done
Q6 done
Q7 done
Q8 done
S3 done
S4 done
RS3 done
RS4 done
RS5 done
RS6 done
RS7 done
R8 done
E-Tron done


In [7]:
new = df[df['price'] != 0 ]
new = new[df['milage'] != 0]
new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11082 entries, 27 to 116959
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         11082 non-null  object 
 1   year          11082 non-null  int64  
 2   price         11082 non-null  float64
 3   milage        11082 non-null  int64  
 4   transmission  11082 non-null  object 
 5   fuel_type     11082 non-null  object 
 6   body_style    11082 non-null  object 
 7   engine_size   11082 non-null  float64
 8   doors         11082 non-null  int64  
 9   location      11082 non-null  object 
 10  id            11082 non-null  object 
 11  link          11082 non-null  object 
 12  timestamp     11082 non-null  object 
 13  make          11082 non-null  object 
 14  model         11082 non-null  object 
dtypes: float64(2), int64(3), object(10)
memory usage: 952.4+ KB


In [8]:
new.to_csv('models.csv')

In [10]:
drop = "DROP TABLE cars_clean"
cursor.execute(drop)
cursor.commit()

sql = """
    CREATE TABLE [cars_clean](
        title varchar(225), 
        year int,  
        price float,
        milage int,  
        transmission varchar(50), 
        fuel_type varchar(50), 
        body_style varchar(50), 
        engine_size float,
        doors int,  
        location varchar(50),
        id varchar(50),
        link varchar(250),
        timestamp date,
        make varchar(50),
        model varchar(50)
    )
"""

cursor.execute(sql)
cursor.commit()



In [11]:
# Insert Dataframe into SQL Server:

for index, row in df.iterrows():
     cursor.execute("""INSERT INTO cars_clean (title,year, price,milage,transmission,fuel_type,body_style,engine_size,doors,location,id,link,timestamp,make,model) 
    values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
""", row.title, row.year, row.price, row.milage,row.transmission,row.fuel_type,row.body_style,row.engine_size,row.doors,row.location,row.id,row.link,row.timestamp,row.make,row.model)
conn.commit()
cursor.close()

In [12]:
conn.close()